In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
%gui qt

In [5]:
from spiketag.base import *

In [6]:
from spiketag.view import *

In [11]:
prb = probe(shank_no=1)
prb[0] = np.array([0,1,2])

In [12]:
prb.mapping[0] = np.array([-10,0])
prb.mapping[1] = np.array([10,0])
prb.mapping[2] = np.array([0,10])

In [13]:
prb.fs = 20000.
prb.n_ch = 3
prb.reorder_by_chip=False

In [5]:
mua = MUA('./cell_0109.bin','./cell_0109.spk.bin', prb, binary_radix=0)

2018-06-06 22:44:26,776 - spiketag - INFO - #############  load data  ###################
2018-06-06 22:44:26,779 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2018-06-06 22:44:26,783 - spiketag - INFO - 174000 * 160 points (111360480 bytes) 
2018-06-06 22:44:26,787 - spiketag - INFO - 160 channels with sampling rate of 25000.0000 
2018-06-06 22:44:26,789 - spiketag - INFO - 6.960 secs (0.116 mins) of data
2018-06-06 22:44:26,790 - spiketag - INFO - #############################################


AttributeError: 'probe' object has no attribute 'reorder_by_chip'

In [18]:
mua.pivotal_pos[0]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [19]:
spks = np.fromfile('./cell_0109.spk.bin', dtype=np.int32)

In [20]:
spks[::2]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [21]:
np.allclose(mua.pivotal_pos[0], spks[::2])

True

## Unit test for individual views

In [6]:
wview = wave_view(mua.data, spks=spks)
wview.show()

NameError: name 'mua' is not defined

In [23]:
spk = mua.tospk()

2018-06-06 18:53:36,591 - spiketag - INFO - mua.tospk()
2018-06-06 18:53:37,162 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:37,163 - spiketag - INFO -  


In [24]:
fet = spk.tofet()

2018-06-06 18:53:37,970 - spiketag - INFO - spk._tofet(groupNo=0, method=weighted-pca, ncomp=6, whiten=False)
2018-06-06 18:53:37,972 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:37,973 - spiketag - INFO -  


In [25]:
clu = fet.toclu()

2018-06-06 18:53:40,433 - spiketag - INFO - clustering finished, used 0.615559101105 sec


In [33]:
spkview = spike_view()

In [35]:
spkview.set_data(spk.spk[0], clu[0])

In [36]:
spkview.show()

In [37]:
fetview = scatter_3d_view()

In [38]:
fetview.set_data(fet.fet[0], clu[0])

In [39]:
fetview.show()

In [41]:
ampview = amplitude_view(fs=prb.fs, scale=1)

In [44]:
ampview.set_data(spk.spk[0], clu[0], mua.pivotal_pos[0])

In [45]:
ampview.show()

In [46]:
treeview = ctree_view()

In [47]:
treeview.set_data(clu[0])

In [48]:
treeview.show()

In [49]:
corview = correlogram_view(fs=prb.fs)

In [50]:
corview.set_data(clu[0], mua.pivotal_pos[0])

In [51]:
corview.show()

In [53]:
traceview = trace_view(fs=prb.fs, spklen=19)

In [54]:
traceview.set_data(mua.data, clu[0], mua.pivotal_pos[0])

In [55]:
traceview.show()

## Unit test for QT5 Wrapper

In [14]:
from spiketag.mvc import MainModel, MainView

In [15]:
model = MainModel('./cell_0109.bin', './cell_0109.spk.bin', prb, binary_radix=11)

2018-06-07 15:41:47,529 - spiketag - INFO - load mua data
2018-06-07 15:41:47,549 - spiketag - INFO - #############  load data  ###################
2018-06-07 15:41:47,551 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2018-06-07 15:41:47,552 - spiketag - INFO - 9280040 * 3 points (111360480 bytes) 
2018-06-07 15:41:47,553 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2018-06-07 15:41:47,555 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2018-06-07 15:41:47,556 - spiketag - INFO - #############################################
2018-06-07 15:41:47,714 - spiketag - INFO - processing folder: ./
2018-06-07 15:41:47,719 - spiketag - INFO - raw data have 9309 spks
2018-06-07 15:41:47,720 - spiketag - INFO - ----------------success------------------
2018-06-07 15:41:47,722 - spiketag - INFO -  
2018-06-07 15:41:47,723 - spiketag - INFO - extract spikes from pivital meta data
2018-06-07 15:41:47,724 - spiketag - INFO - mua.tospk()
2018-06-07 15:41:47,734 - s

In [16]:
view = MainView(prb)

In [17]:
view.show()

In [18]:
view.set_data(0, model.mua, model.spk[0], model.fet[0], model.clu[0])

In [42]:
view.traceview.locate_buffer += 30

In [13]:
thr = -31.00

In [14]:
idx = np.where(model.spk[0].min(axis=1).min(axis=1)>thr)[0]

In [15]:
idx.shape

(5032,)

In [16]:
model.mua.spk_times[0] = np.delete(model.mua.spk_times[0], idx, axis=0)

In [17]:
model.spk[0] = np.delete(model.spk[0], idx, axis=0)

In [18]:
model.fet[0] = model.spk._tofet(0, method='pca')

In [19]:
model.clu[0] = model.fet._toclu(0)

In [20]:
model.cluster(group_id=0, method='hdbscan', fall_off_size=30)

In [13]:
@view.ampview.clip.connect
def on_clip(thres):
    clip_idx = np.where(view.ampview.poses[:,1] > thres)[0]
#     print(clip_idx.shape)
    model.mua.spk_times[0] = np.delete(model.mua.spk_times[0], clip_idx, axis=0)
    model.spk[0] = np.delete(model.spk[0], clip_idx, axis=0)
    model.fet[0] = model.spk.tofet(group_id=0, method='pca')
    model.cluster(group_id=0, method='hdbscan', fall_off_size=15)

2018-06-07 11:28:24,572 - spiketag - INFO - spk._tofet(group_id=0, method=pca, ncomp=6, whiten=False)
2018-06-07 11:28:24,573 - spiketag - INFO - ----------------success------------------
2018-06-07 11:28:24,575 - spiketag - INFO -  
  File "/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/anaconda2/lib/python2.7/site-packages/tornado/ioloop.

In [42]:
model.spk[0].shape

(1678, 19, 3)

In [39]:
model.fet = model.spk.tofet()

2018-06-07 11:33:46,742 - spiketag - INFO - spk._tofet(group_id=0, method=weighted-pca, ncomp=6, whiten=False)
2018-06-07 11:33:46,746 - spiketag - INFO - ----------------success------------------
2018-06-07 11:33:46,747 - spiketag - INFO -  


In [40]:
model.fet[0].shape

(1678, 6)

In [41]:
model.clu = model.fet.toclu()

2018-06-07 11:33:49,754 - spiketag - INFO - clustering finished, used 0.066253900528 sec


In [19]:
model.mua.spk_times[0].shape

(4740,)

In [16]:
view.ampview.poses[:, 1].min()

-146.53759765625

array([-19.429688, -18.166504, -17.284668, ..., -17.5     , -19.528809,
       -28.05664 ], dtype=float32)

In [14]:
model.mua.spk_times[0].shape

(9309,)

In [15]:
model.spk[0].shape

(9309, 19, 3)

In [16]:
model.fet[0].shape

(9309, 6)

In [28]:
import sys
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *

class sorter(QWidget):

    def __init__(self):
        super(sorter, self).__init__()
        self.initUI()

    def initUI(self):

        hbox = QHBoxLayout(self)
        self.splitter1 = QSplitter(Qt.Horizontal)
#         textedit = QTextEdit()
#         self.splitter1.addWidget(self.topleft)
#         self.splitter1.addWidget(textedit)
#         self.splitter1.setSizes([100,200])
        self.splitter2 = QSplitter(Qt.Horizontal)
        self.splitter_fet = QSplitter(Qt.Vertical)

        self.splitter3 = QSplitter(Qt.Vertical)
        self.splitter3.addWidget(self.splitter1)
        self.splitter3.addWidget(self.splitter2)
#         self.splitter2.addWidget(self.bottom)

        hbox.addWidget(self.splitter3)

        self.setLayout(hbox)
        QApplication.setStyle(QStyleFactory.create('Cleanlooks'))

        self.setGeometry(300, 300, 300, 200)
        self.setWindowTitle('spiketag')
#         self.show()

    def set_data(self, mua, spk, fet, clu):
        ### init view and set_data
        self.spkview = spike_view()
        self.spkview.set_data(spk, clu)
        self.fetview0 = scatter_3d_view()
        self.fetview0.set_data(fet[:,[0,1,2]].copy(), clu)
        self.fetview1 = scatter_3d_view()
        self.fetview1.set_data(fet[:,[1,3,4]].copy(), clu)
        self.ampview = amplitude_view(fs=fs, scale=1)
        self.ampview.set_data(spk, clu, mua.pivotal_pos[0])
        self.treeview = ctree_view()
        self.treeview.set_data(clu)
        self.corview = correlogram_view(fs=fs)
        self.corview.set_data(clu, mua.pivotal_pos[0])
        self.traceview = trace_view(fs=mua.fs, spklen=spk.shape[1])
        self.traceview.set_data(mua.data, clu, mua.pivotal_pos[0])
#         self.traceview.locate_buffer = 2000
        
        ### put views into splitter
        self.splitter1.addWidget(self.traceview.native)
        self.splitter1.addWidget(self.splitter_fet)
        self.splitter_fet.addWidget(self.fetview0.native)
        self.splitter_fet.addWidget(self.fetview1.native)
        self.splitter1.addWidget(self.spkview.native)
        
        self.splitter2.addWidget(self.corview.native)
        self.splitter2.addWidget(self.treeview.native)
        self.splitter2.addWidget(self.ampview.native)


In [29]:
%gui qt

In [30]:
ex = sorter()

In [31]:
spk = mua.tospk()
fet = spk.tofet(method='pca', ncomp=9, whiten=False)
clu = fet.toclu(method='hdbscan')

2018-06-06 18:53:54,900 - spiketag - INFO - mua.tospk()
2018-06-06 18:53:54,909 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:54,910 - spiketag - INFO -  
2018-06-06 18:53:54,943 - spiketag - INFO - spk._tofet(groupNo=0, method=pca, ncomp=9, whiten=False)
2018-06-06 18:53:54,945 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:54,946 - spiketag - INFO -  
2018-06-06 18:53:55,855 - spiketag - INFO - clustering finished, used 0.906870126724 sec


In [32]:
ex.set_data(mua, spk.spk[1], fet.fet[1], clu[1])

KeyError: 1

In [66]:
ex.show()

In [70]:
ex.traceview.locate_buffer = 300

In [97]:
ex.fetview0.set_data(fet.fet[1][:,[1,3,4]], clu[1])
ex.update()

ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4774b690>> repeat 8192
ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4a0aa110>> repeat 8192


In [20]:
gclu = clu[1]

In [21]:
gclu.selectlist

array([ 848, 2681, 2795, 2899, 2951, 3162, 3521, 4018, 4031, 4074, 4169,
       4224, 4237, 4486, 4489, 4648, 5064, 5078, 5823, 5966, 5967, 6047,
       6049, 6070, 6610, 7207, 7319, 7919, 8057, 8167, 8378, 8602, 8679,
       9240])

2018-03-11 15:08:33,804 - spiketag - DEBUG - ctree view expand clustier 9311 here
2018-03-11 15:08:51,809 - spiketag - DEBUG - ctree view expand clustier 9313 here


In [21]:
gclu._registered_func_name('select')

['spiketag.view.spike_view.on_select_id121320795744',
 'spiketag.view.scatter_3d_view.on_select_id121444857928',
 'spiketag.view.scatter_3d_view.on_select_id121449105600',
 'spiketag.view.amplitude_view.on_select_id121452650320',
 'spiketag.view.trace_view.on_select_id121529220464']

2018-03-11 14:14:28,666 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:29,401 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:31,183 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:32,756 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,080 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,932 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:35,486 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,014 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,999 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:37,535 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:39,278 - spiketag - DEBUG - ctree view expand clustier 9312 here
2018-03-11 14:14:43,798 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-